## Multivariate Linear Regression 

Hi, I'm a total noob and have attempted to fit the life expctancy data on a linear regression model without using sklearn. My attempts at predicting for x_train is confusing(bottom) , therefore if anyone passing by knows a bit about how to train the x_test to my model please do shine light.:p thank you in advance. (my prediction for y just changes in range with changing theta and I dont quite understand how to get more accurate predictions.did my due diligence but hitting a wall.

### Data load and EDA

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
print('Modules are imported.')

In [ ]:
df=pd.read_csv('/kaggle/input/life-expectancy-who/Life Expectancy Data.csv')
df.head()

In [ ]:
#make country column as the Index of the df
df.set_index("Country",inplace= True)
df.head()

In [ ]:
df.shape

In [ ]:
df['Year'].unique

In [ ]:
df.columns

In [ ]:
#drop columns 
useless_col = ['percentage expenditure','Hepatitis B','Status',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources']
df.drop(useless_col,axis=1,inplace=True)


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# find all the null values in df
df.isnull().sum() 

In [ ]:
df.Alcohol.value_counts() 

In [ ]:
#replace alcohol null values with the most recurrent number
df.Alcohol.fillna(0.01,inplace=True)

In [ ]:
#replace GDP null values with the median
b=df.GDP.median()
df.GDP.fillna(b,inplace=True)

In [ ]:
#the rest drop the null values
df.dropna(inplace=True)

In [ ]:
df.isnull().sum() 

In [ ]:
#correlation scores
df.corr()

In [ ]:
#correlation depicted in heatmap
sns.heatmap(df.corr())

In [ ]:
#a scatterplot overview of the correlations between features
sns.pairplot(df)

### Linear Regression 


Linear regression predicts the dependent value (y) according to the independent variable (x). The output here is the dependent value, and the input is the independent value.

$
\begin{equation}
y = b + w_1.x_1 + w_2.x_2
\end{equation}
$

In a vector form:

$
\begin{equation}
y = b + (w_1 w_2).\binom{x_1}{x_2}
\end{equation}
$

Where
$
\begin{equation}
W = (w_1 w_2)
\end{equation}
$
and
$
\begin{equation}
X = \binom{x_1}{x_2}
\end{equation}
$

### Cost Function


The loss over **m** examples:
Mean Squared Error:


$
\begin{equation}
J = \frac{1}{2m} \sum_{i=1}^{m} (y - \hat{y})^2
\end{equation}
$

The objective of the gradient descent algorithm is to minimize this error  value.

Gradient Descent Objective is to 
$
\begin{equation}
min(J)
\end{equation}
$





### The gradient descent algorithm:


do it for all the theta values(no of theta values = no of features + 1).

![Loss vs Param](GD.png)




In [ ]:
X = df.values[:, 2:7]  #  input values from last 5 columns
y = df.values[:, 1]  # get output values 2nd coulmn(life_exp)
m = len(y) # Number of training examples

print('Total no of training examples (m) = %s \n' %(m))

# Show only first 5 records
for i in range(5):
    print('x =', X[i, ], ', y =', y[i])

### Then we Normalize the features because they all are not in the same range of measurement:

Feature Scaling: In feature scaling we divide the input value by range(max - min) of input variable. By this technique we get new range of just 1.

Mean Normalization: In mean normalization we subtract the average value from the input variable and then divide it by range(max - min) or by standard deviation of input variable.

In [ ]:
def feature_normalize(X):
  """
    Normalizes the features(input variables) in X.

    Parameters
    ----------
    X : n dimensional array (matrix), shape (n_samples, n_features)
        Features(input varibale) to be normalized.

    Returns
    -------
    X_norm : n dimensional array (matrix), shape (n_samples, n_features)
        A normalized version of X.
    mu : n dimensional array (matrix), shape (n_features,)
        The mean value.
    sigma : n dimensional array (matrix), shape (n_features,)
        The standard deviation.
  """
  # mean of column, hence axis = 0
  mu = np.mean(X, axis = 0)  
  # Notice the parameter ddof (Delta Degrees of Freedom)  value is 1
  sigma = np.std(X, axis= 0, ddof = 1)  # Standard deviation (can also use range)
  X_norm = (X - mu)/sigma
  return X_norm, mu, sigma

In [ ]:
X, mu, sigma = feature_normalize(X)

print('mu= ', mu)
print('sigma= ', sigma)
print('X_norm= ', X[:5])

In [ ]:
#New mean or avearage value of normalized X feature is 0
mu_testing = np.mean(X, axis = 0) # mean
mu_testing

In [ ]:
#New range or standard deviation of normalized X feature is 1
sigma_testing = np.std(X, axis = 0, ddof = 1) # mean
sigma_testing

In [ ]:
# Lets use hstack() function from numpy to add column of ones to X feature 
# This will be our final X matrix (feature matrix)
X = np.hstack((np.ones((m,1)), X))
X[:5]

In [ ]:
from sklearn.model_selection import train_test_split
X, x_test, y, y_test = train_test_split(X,y,test_size =0.2)

In [ ]:
X[1]

In [ ]:
x_test[1]

In [ ]:
def compute_cost(X, y, theta):
  """
  Compute the cost of a particular choice of theta for linear regression.

  Input Parameters
  ----------------
  X : 2D array where each row represent the training example and each column represent the feature ndarray. Dimension(m x n)
      m= number of training examples
      n= number of features (including X_0 column of ones)
  y : 1D array of labels/target value for each traing example. dimension(1 x m)

  theta : 1D array of fitting parameters or weights. Dimension (1 x n)

  Output Parameters
  -----------------
  J : Scalar value.
  """
  predictions = X.dot(theta)
  #print('predictions= ', predictions[:5])
  errors = np.subtract(predictions, y)
  #print('errors= ', errors[:5]) 
  sqrErrors = np.square(errors)
  #print('sqrErrors= ', sqrErrors[:5]) 
  #J = 1 / (2 * m) * np.sum(sqrErrors)
  # OR
  # We can merge 'square' and 'sum' into one by taking the transpose of matrix 'errors' and taking dot product with itself
  # If your confuse about this try to do this with few values for better understanding  
  J = 1/(2 * m) * errors.T.dot(errors)

  return J

In [ ]:
def gradient_descent(X, y, theta, alpha, iterations):
  """
  Compute cost for linear regression.

  Input Parameters
  ----------------
  X : 2D array where each row represent the training example and each column represent the feature ndarray. Dimension(m x n)
      m= number of training examples
      n= number of features (including X_0 column of ones)
  y : 1D array of labels/target value for each traing example. dimension(m x 1)
  theta : 1D array of fitting parameters or weights. Dimension (1 x n)
  alpha : Learning rate. Scalar value
  iterations: No of iterations. Scalar value. 

  Output Parameters
  -----------------
  theta : Final Value. 1D array of fitting parameters or weights. Dimension (1 x n)
  cost_history: Conatins value of cost for each iteration. 1D array. Dimansion(m x 1)
  """
  cost_history = np.zeros(iterations)

  for i in range(iterations):
    predictions = X.dot(theta)
    #print('predictions= ', predictions[:5])
    errors = np.subtract(predictions, y)
    #print('errors= ', errors[:5])
    sum_delta = (alpha / m) * X.transpose().dot(errors);
    #print('sum_delta= ', sum_delta[:5])
    theta = theta - sum_delta;

    cost_history[i] = compute_cost(X, y, theta)  

  return theta, cost_history

In [ ]:
# We need theta parameter for every input variable. since we have 6 input variable including X_0 (column of ones)
theta = np.zeros(6)
iterations = 400;
alpha = 0.15;

In [ ]:
theta, cost_history = gradient_descent(X, y, theta, alpha, iterations)
print('Final value of theta =', theta)
print('First 5 values from cost_history =', cost_history[:5])
print('Last 5 values from cost_history =', cost_history[-5 :])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1, iterations +1), cost_history, color ='blue')
plt.rcParams["figure.figsize"] = (10,6)
plt.grid()
plt.xlabel("Number of iterations")
plt.ylabel("cost (J)")
plt.title("Convergence of gradient descent")

### Effect Of Learning Rate On Convergence

In [ ]:
iterations = 400;
theta = np.zeros(6)

alpha = 0.005;
theta_1, cost_history_1 = gradient_descent(X, y, theta, alpha, iterations)

alpha = 0.01;
theta_2, cost_history_2 = gradient_descent(X, y, theta, alpha, iterations)

alpha = 0.02;
theta_3, cost_history_3 = gradient_descent(X, y, theta, alpha, iterations)

alpha = 0.03;
theta_4, cost_history_4 = gradient_descent(X, y, theta, alpha, iterations)

alpha = 0.15;
theta_5, cost_history_5 = gradient_descent(X, y, theta, alpha, iterations)

plt.plot(range(1, iterations +1), cost_history_1, color ='black', label = 'alpha = 0.005')
plt.plot(range(1, iterations +1), cost_history_2, color ='blue', label = 'alpha = 0.01')
plt.plot(range(1, iterations +1), cost_history_3, color ='green', label = 'alpha = 0.02')
plt.plot(range(1, iterations +1), cost_history_4, color ='brown', label = 'alpha = 0.03')
plt.plot(range(1, iterations +1), cost_history_5, color ='blue', label = 'alpha = 0.15')

plt.rcParams["figure.figsize"] = (10,6)
plt.grid()
plt.xlabel("Number of iterations")
plt.ylabel("cost (J)")
plt.title("Effect of Learning Rate On Convergence of Gradient Descent")
plt.legend()


In [ ]:
iterations = 100;
theta = np.zeros(6)

alpha = 1.32;
theta_6, cost_history_6 = gradient_descent(X, y, theta, alpha, iterations)

plt.plot(range(1, iterations +1), cost_history_6, color ='brown')
plt.rcParams["figure.figsize"] = (10,6)
plt.grid()
plt.xlabel("Number of iterations")
plt.ylabel("cost (J)")
plt.title("Effect of Large Learning Rate On Convergence of Gradient Descent")

In [ ]:


for i in range(50):
    hu = (np.array(x_test[i]))
    ki=hu[1:6]
    normalize_test_data = ((ki - mu) / sigma)
    normalize_test_data = np.hstack((np.ones(1), normalize_test_data))
    y_pred = normalize_test_data.dot(theta_3)
    print('Predicted age : {},real y value: {}'.format(y_pred, y_test[i]) )
    
    

In [ ]:

for i in range(50):
    hu = (np.array(x_test[i]))
    ki=hu[1:6]
    normalize_test_data = ((ki - mu) / sigma)
    normalize_test_data = np.hstack((np.ones(1), normalize_test_data))
    y_pred = normalize_test_data.dot(theta_1)
    print('Predicted age : {},real y value: {}'.format(y_pred, y_test[i]) )

In [ ]:

for i in range(50):
    hu = (np.array(x_test[i]))
    ki=hu[1:6]
    normalize_test_data = ((ki - mu) / sigma)
    normalize_test_data = np.hstack((np.ones(1), normalize_test_data))
    y_pred = normalize_test_data.dot(theta_2)
    print('Predicted age : {},real y value: {}'.format(y_pred, y_test[i]) )